## Get daily COVID19 report data as tidy data using the covid19.mathdro.id API

The covid19.mathdro.id API (documentation at https://github.com/mathdroid/covid-19-api ) can return ALL daily reports using this method:

- `/api/daily/[date]`: detail of updates in a [date] (e.g. /api/daily/2-14-2020)

What we'd like to do is collect ALL of the reports, and compile into "tidy data" format that we can then work with.



### Set up Python libraries and API base

In [0]:
import requests
import pandas as pd
from datetime import datetime

API_BASE_URL = 'https://covid19.mathdro.id/api'

### Get daily case summaries

We first need to know which dates we can request reports from.  We're going to use the result of the `/daily` API method just to get the range of dates available:

In [0]:
daily_cases = requests.get(API_BASE_URL + '/daily').json()

Go through the cases and pull out the `reportDateString` value, adding these to a Python list as we go along.

In [0]:
day_data = []
for day_cases in daily_cases:
  # day_cases['reportDateString'] = pd.to_datetime(day_cases['reportDateString'])
  day_data.append(day_cases['reportDateString'])

This is what our list looks like:

In [147]:
day_data[1:10]

['2020/01/21',
 '2020/01/22',
 '2020/01/23',
 '2020/01/24',
 '2020/01/25',
 '2020/01/26',
 '2020/01/27',
 '2020/01/28',
 '2020/01/29']

### Request daily detail for each day

Using the `daily/[date]` API method, and our list of dates, we get a list of daily detail reports (one for each location with a report for that day).  This entails 1 API call per day available.  We'll store these report lists in a list which we'll call `all_days_detail`.

Note that we have to convert date formats to make this work.

In [135]:
all_days_detail = []
for day in day_data:
  # Convert 2020/12/01 to 12-1-2020 without the zeroes
  day_as_datetime = datetime.strptime(day, '%Y/%m/%d')
  date_day = day_as_datetime.day
  date_month = day_as_datetime.month
  date_year = day_as_datetime.year
  day_with_dashes = '%d-%d-%d' % (date_month, date_day, date_year)
  print(API_BASE_URL + '/daily/' + day_with_dashes)
  day_detail = requests.get(API_BASE_URL + '/daily/' + day_with_dashes).json()
  all_days_detail.append(day_detail)

https://covid19.mathdro.id/api/daily/1-20-2020
https://covid19.mathdro.id/api/daily/1-21-2020
https://covid19.mathdro.id/api/daily/1-22-2020
https://covid19.mathdro.id/api/daily/1-23-2020
https://covid19.mathdro.id/api/daily/1-24-2020
https://covid19.mathdro.id/api/daily/1-25-2020
https://covid19.mathdro.id/api/daily/1-26-2020
https://covid19.mathdro.id/api/daily/1-27-2020
https://covid19.mathdro.id/api/daily/1-28-2020
https://covid19.mathdro.id/api/daily/1-29-2020
https://covid19.mathdro.id/api/daily/1-30-2020
https://covid19.mathdro.id/api/daily/1-31-2020
https://covid19.mathdro.id/api/daily/2-1-2020
https://covid19.mathdro.id/api/daily/2-2-2020
https://covid19.mathdro.id/api/daily/2-3-2020
https://covid19.mathdro.id/api/daily/2-4-2020
https://covid19.mathdro.id/api/daily/2-5-2020
https://covid19.mathdro.id/api/daily/2-6-2020
https://covid19.mathdro.id/api/daily/2-7-2020
https://covid19.mathdro.id/api/daily/2-8-2020
https://covid19.mathdro.id/api/daily/2-9-2020
https://covid19.mathdr

Let's see what one day's report list looks like.  We observe that it's a list of JSON blocks.

In [149]:
all_days_detail[4]

[{'confirmed': '549',
  'countryRegion': 'Mainland China',
  'deaths': '24',
  'lastUpdate': Timestamp('2020-01-24 17:00:00'),
  'provinceState': 'Hubei',
  'recovered': '31'},
 {'confirmed': '53',
  'countryRegion': 'Mainland China',
  'deaths': '',
  'lastUpdate': Timestamp('2020-01-24 17:00:00'),
  'provinceState': 'Guangdong',
  'recovered': '2'},
 {'confirmed': '43',
  'countryRegion': 'Mainland China',
  'deaths': '',
  'lastUpdate': Timestamp('2020-01-24 17:00:00'),
  'provinceState': 'Zhejiang',
  'recovered': '1'},
 {'confirmed': '36',
  'countryRegion': 'Mainland China',
  'deaths': '',
  'lastUpdate': Timestamp('2020-01-24 17:00:00'),
  'provinceState': 'Beijing',
  'recovered': '1'},
 {'confirmed': '27',
  'countryRegion': 'Mainland China',
  'deaths': '',
  'lastUpdate': Timestamp('2020-01-24 17:00:00'),
  'provinceState': 'Chongqing',
  'recovered': ''},
 {'confirmed': '24',
  'countryRegion': 'Mainland China',
  'deaths': '',
  'lastUpdate': Timestamp('2020-01-24 17:00:0

### Create a data frame with one report per row

Iterate through the list of lists, and add each report to a PANDAS data frame.

Note that early reports seemed to use a key name of `\ufeffprovinceState` so where we detect that, we convert it to `provinceState`.

In [0]:
df = pd.DataFrame(columns = ['provinceState', 'countryRegion', 'lastUpdate', 'confirmed', 'deaths', 'recovered'])
#for day_detail in all_days_detail:
for day_detail in all_days_detail:
  for province_detail in day_detail:
    if '\ufeffprovinceState' in province_detail:
      province_detail['provinceState'] = province_detail.get('\ufeffprovinceState')
      del province_detail['\ufeffprovinceState']

    province_detail['lastUpdate'] = pd.to_datetime(province_detail['lastUpdate'])
    df = df.append(province_detail, ignore_index = True)


Let's take a look at some data in the middle of the data frame:

In [138]:
df.tail(1000).head(100)

,provinceState,countryRegion,lastUpdate,confirmed,deaths,recovered,latitude,longitude
4153,"Fairfax County, VA",US,2020-03-08 21:33:02,2,0,0,38.9085,-77.2405
4154,"Hillsborough, FL",US,2020-03-03 18:33:02,2,0,0,27.9904,-82.3018
4155,"Jackson County, OR",US,2020-03-08 00:13:16,2,0,0,42.3345,-122.7647
4156,"Lee County, FL",US,2020-03-08 15:33:04,2,1,0,26.6630,-81.9535
4157,"Pinal County, AZ",US,2020-03-07 17:13:23,2,0,0,32.8162,-111.2845
...,...,...,...,...,...,...,...,...
4248,,Italy,2020-03-09 18:13:11,9172,463,724,43.0000,12.0000
4249,,South Korea,2020-03-09 09:03:03,7478,53,118,36.0000,128.0000
4250,,Iran,2020-03-09 10:53:03,7161,237,2394,32.0000,53.0000
4251,Guangdong,Mainland China,2020-03-09 01:43:01,1352,8,1260,23.3417,113.4244


We can now do things like subsetting, etc.

In [140]:
df[df['countryRegion'] == 'US']

,provinceState,countryRegion,lastUpdate,confirmed,deaths,recovered,latitude,longitude
31,Washington,US,2020-01-22 17:00:00,1,,,NaN,NaN
69,Washington,US,2020-01-23 17:00:00,1,,,NaN,NaN
117,Washington,US,2020-01-24 17:00:00,1,,,NaN,NaN
118,Chicago,US,2020-01-24 17:00:00,1,,,NaN,NaN
158,Washington,US,2020-01-25 17:00:00,1,,,NaN,NaN
...,...,...,...,...,...,...,...,...
5142,Wyoming,US,2020-03-12 03:13:09,1,0,0,42.7560,-107.3025
5148,Alaska,US,2020-03-10 02:33:04,0,0,0,61.3707,-152.4044
5149,Idaho,US,2020-03-10 02:33:04,0,0,0,44.2405,-114.4788
5150,Maine,US,2020-03-10 02:33:04,0,0,0,44.6939,-69.3819


### Write out the data to a CSV file

In [0]:
df.to_csv('reports.csv', index=None)

In [0]:
# If you're in a Google Colab notebook, you'll need to this in order to download the report to your computer:

# from google.colab import files
# files.download('reports.csv')